In [ ]:
# 12.1. MNIST 데이터 입력

import numpy as np 
from keras.datasets import mnist
(x_train, t_train), (x_test, t_test) = mnist.load_data()
t_trainlbl, t_testlbl = t_train, t_test

# 28X28 을 784 로 수정
x_train = x_train.reshape(60000,784)    # 주석 (1)
x_test = x_test.reshape(10000,784)    

# one-hot label 
T0 = np.zeros((t_train.size, 10))    #(60000,10) = 000
T1 = np.zeros((t_test.size, 10))    #(10000,10) = 000

for idx in range(t_train.size): T0[idx][t_train[idx]] = 1    #(3))
for idx in range(t_test.size): T1[idx][t_test[idx]] = 1

t_train, t_test = T0, T1

# normalize 0.0 ~ 1.0
x_train = x_train / 255
x_test = x_test / 255

print('MNIST DataSets 준비 완료')


In [ ]:
# 12.2. 함수정의

# 미분함수 
def numerical_diff(f, x):
    h = 1e-4    # 0.0001
    nd_coef = np.zeros_like(x)
    it = np.nditer(x, flags=['multi_index'], op_flags=['readwrite'])
    while not it.finished:
        index = it.multi_index
        tmp = x[index]
        x[index] = tmp + h
        fxh2 = f()    # f(x+h)
        x[index] = tmp - h 
        fxh1 = f()    # f(x-h)
        nd_coef[index] = (fxh2 - fxh1) / (2*h)
        x[index] = tmp 
        it.iternext()
    return nd_coef

# 시그모이드
def sigmoid(x):
    return 1 / (1+np.exp(-x))

# 소프트맥스
def softmax(x):
    if x.ndim == 1:  # 기본 1개 처리과정 , 벡터입력
        x = x - np.max(x) 
        return np.exp(x) / np.sum(np.exp(x))
    if x.ndim == 2:  # 배치용 n 개 처리, 행렬입력
        x = x.T - np.max(x.T, axis=0)
        return (np.exp(x) / np.sum(np.exp(x), axis=0)).T

# 크로스엔트로피오차
def cee(y, t):
    if y.ndim == 1:
        t = t.reshape(1, t.size)  # 크기가 1xN 인 2차원 행렬로 재구성
        y = y.reshape(1, y.size)
    result = -np.sum(t * np.log(y + 1e-7))  / y.shape[0] 
    return result 
